Meta data of the top 100 tech companies

In [21]:
import pandas as pd

# -------------------------------
# STEP 1: LOAD REQUIRED DATA
# -------------------------------

# File paths
marketcap_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/companiesmarketcap.com - Largest tech companies by market cap.csv"
currency_info_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_exchange_currency_info.csv"
metadata_output_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_metadata_top100.csv"

# Load datasets
marketcap_df = pd.read_csv(marketcap_path)
currency_info_df = pd.read_csv(currency_info_path)

print("✅ Marketcap and Currency Info datasets loaded successfully.")

# -------------------------------
# STEP 2: SELECT TOP 100 COMPANIES
# -------------------------------

# Select the top 100 companies
marketcap_top100 = marketcap_df.head(100)[['Rank', 'Name', 'Symbol']].drop_duplicates(subset='Symbol')

# Check for missing essential columns in currency info
required_currency_columns = ['Symbol', 'Exchange', 'Currency', 'Country']
for col in required_currency_columns:
    if col not in currency_info_df.columns:
        print(f"⚠️ Column '{col}' is missing in currency_info dataset. Adding default value 'Unknown'.")
        currency_info_df[col] = 'Unknown'

# -------------------------------
# STEP 3: MERGE DATASETS
# -------------------------------

# Merge top 100 companies with currency and exchange info
metadata_top100 = marketcap_top100.merge(
    currency_info_df[['Symbol', 'Exchange', 'Currency', 'Country']],
    on='Symbol',
    how='left'
)

# Fill missing values with 'Unknown'
metadata_top100[['Exchange', 'Currency', 'Country']] = metadata_top100[['Exchange', 'Currency', 'Country']].fillna('Unknown')

# -------------------------------
# STEP 4: SAVE METADATA FILE
# -------------------------------

# Save metadata for top 100 companies to CSV
metadata_top100.to_csv(metadata_output_path, index=False)
print(f"✅ Metadata for top 100 companies saved to {metadata_output_path}")


✅ Marketcap and Currency Info datasets loaded successfully.
✅ Metadata for top 100 companies saved to /Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_metadata_top100.csv


data without currency changing

In [23]:
import pandas as pd
import numpy as np
import os
import glob

# -------------------------------
# STEP 1: LOAD REQUIRED DATA
# -------------------------------

# File paths
metadata_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_metadata_top100.csv"
forex_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/forex_rates.csv"
data_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Data_of_biggest_100_companies/*.csv"
timeseries_output_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_timeseries_top100.csv"

# Load datasets
metadata_df = pd.read_csv(metadata_path)
forex_df = pd.read_csv(forex_path, index_col=0, parse_dates=True)

print("✅ Metadata and Forex data loaded successfully.")

# -------------------------------
# STEP 2: LOAD AND COMBINE TIME SERIES DATA
# -------------------------------

# Load stock data files for top 100 symbols
all_files = glob.glob(data_path)
combined_data = []

# Filter only top 100 symbols
top100_symbols = metadata_df['Symbol'].unique()

for file in all_files:
    symbol = os.path.basename(file).split('_')[0]
    if symbol in top100_symbols:
        df = pd.read_csv(file)
        df['Symbol'] = symbol
        df = df[['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'Symbol']]
        combined_data.append(df)

# Combine all data into one DataFrame
timeseries_df = pd.concat(combined_data, ignore_index=True)
timeseries_df['Date'] = pd.to_datetime(timeseries_df['Date'], errors='coerce')

# Merge currency info from metadata
timeseries_df = timeseries_df.merge(
    metadata_df[['Symbol', 'Currency']],
    on='Symbol',
    how='left'
)

# -------------------------------
# STEP 3: APPLY CURRENCY CONVERSION
# -------------------------------

# Add USD columns
for col in ['Adj Close', 'Close', 'High', 'Low', 'Open']:
    timeseries_df[f'USD_{col}'] = timeseries_df[col]

# Convert non-USD values
non_usd_mask = timeseries_df['Currency'] != 'USD'
non_usd_df = timeseries_df[non_usd_mask]

if not non_usd_df.empty:
    for currency in non_usd_df['Currency'].unique():
        forex_pair = f"{currency}USD=X"
        if forex_pair in forex_df.columns:
            for date, group in non_usd_df.groupby('Date'):
                if date in forex_df.index:
                    conversion_rate = forex_df.at[date, forex_pair]
                    mask = (timeseries_df['Date'] == date) & (timeseries_df['Currency'] == currency)
                    for col in ['Adj Close', 'Close', 'High', 'Low', 'Open']:
                        usd_col = f'USD_{col}'
                        timeseries_df.loc[mask, usd_col] = timeseries_df.loc[mask, col] * conversion_rate

print("✅ Currency conversion applied successfully.")

# -------------------------------
# STEP 4: SAVE TIME SERIES FILE
# -------------------------------

# Select final columns
final_columns = ['Date', 'Symbol', 'USD_Adj Close', 'USD_Close', 'USD_High', 'Low', 'USD_Low', 'USD_Open', 'Volume']
timeseries_df = timeseries_df[final_columns]

# Save the final time series data
timeseries_df.to_csv(timeseries_output_path, index=False)
print(f"✅ Time series data for top 100 companies saved to {timeseries_output_path}")


/var/folders/g1/lmg3w4rs5t7f4c_vswsjfrz80000gn/T/ipykernel_7916/3045492590.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forex_df = pd.read_csv(forex_path, index_col=0, parse_dates=True)


✅ Metadata and Forex data loaded successfully.
✅ Currency conversion applied successfully.
✅ Time series data for top 100 companies saved to /Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_timeseries_top100.csv


In [31]:
import pandas as pd
import numpy as np
import os
import glob

# -------------------------------
# STEP 1: LOAD REQUIRED DATA
# -------------------------------

# File paths
metadata_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_metadata_top100.csv"
forex_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/forex_rates.csv"
currency_info_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_exchange_currency_info.csv"
data_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Data_of_biggest_100_companies/*.csv"
timeseries_output_path = "/Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_timeseries_top100.csv"

# Load datasets
metadata_df = pd.read_csv(metadata_path)
forex_df = pd.read_csv(forex_path, index_col=0, parse_dates=True)
currency_info_df = pd.read_csv(currency_info_path)

print("✅ Metadata, Forex, and Currency Info data loaded successfully.")

# -------------------------------
# STEP 2: FORMAT DATES AND CLEAN DATA
# -------------------------------

# Tarih formatını standart hale getir
forex_df.index = pd.to_datetime(forex_df.index, errors='coerce').date
metadata_df['Symbol'] = metadata_df['Symbol'].astype(str)

# -------------------------------
# STEP 3: LOAD AND COMBINE TIME SERIES DATA
# -------------------------------

# Load stock data files for top 100 symbols
all_files = glob.glob(data_path)
combined_data = []

# Filter only top 100 symbols
top100_symbols = metadata_df['Symbol'].unique()

for file in all_files:
    symbol = os.path.basename(file).split('_')[0]
    if symbol in top100_symbols:
        df = pd.read_csv(file)
        df['Symbol'] = symbol
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
        df = df[['Symbol', 'Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']]
        combined_data.append(df)

# Combine all data into one DataFrame
timeseries_df = pd.concat(combined_data, ignore_index=True)
timeseries_df['Date'] = pd.to_datetime(timeseries_df['Date'], errors='coerce').dt.date
timeseries_df.sort_values(by=['Date', 'Symbol'], inplace=True)
timeseries_df.reset_index(drop=True, inplace=True)

# Merge currency info from metadata
timeseries_df = timeseries_df.merge(
    metadata_df[['Symbol', 'Currency']],
    on='Symbol',
    how='left'
)

# -------------------------------
# STEP 4: CLEAN AND VALIDATE DATA TYPES
# -------------------------------

# Sayısal sütunları float türüne dönüştür
for col in ['Adj Close', 'Close', 'High', 'Low', 'Open']:
    timeseries_df[col] = pd.to_numeric(timeseries_df[col], errors='coerce')

# USD sütunlarını baştan oluştur
for col in ['Adj Close', 'Close', 'High', 'Low', 'Open']:
    timeseries_df[f'USD_{col}'] = np.nan

# -------------------------------
# STEP 5: APPLY CURRENCY CONVERSION
# -------------------------------

# Döviz dönüşümü
for currency, forex_pair in {
    'KRW': 'KRWUSD=X',
    'HKD': 'HKDUSD=X',
    'EUR': 'EURUSD=X',
    'JPY': 'JPYUSD=X',
    'ILS': 'ILSUSD=X',
    'TWD': 'TWDUSD=X',
    'CAD': 'CADUSD=X',
    'THB': 'THBUSD=X'
}.items():
    if forex_pair in forex_df.columns:
        valid_dates = forex_df.index.intersection(timeseries_df['Date'].dropna().unique())
        conversion_rates = forex_df.loc[valid_dates, forex_pair].to_dict()
        
        for date in valid_dates:
            mask = (timeseries_df['Date'] == date) & (timeseries_df['Currency'] == currency)
            conversion_rate = conversion_rates.get(date)
            
            if pd.notnull(conversion_rate):
                for col in ['Adj Close', 'Close', 'High', 'Low', 'Open']:
                    usd_col = f'USD_{col}'
                    timeseries_df.loc[mask, usd_col] = timeseries_df.loc[mask, col].astype(float) * float(conversion_rate)

# USD olan değerleri doğrudan taşı
usd_mask = timeseries_df['Currency'] == 'USD'
for col in ['Adj Close', 'Close', 'High', 'Low', 'Open']:
    usd_col = f'USD_{col}'
    timeseries_df.loc[usd_mask, usd_col] = timeseries_df.loc[usd_mask, col]

print("✅ Currency conversion applied successfully.")

# -------------------------------
# STEP 6: FINAL FORMATTING & SAVE
# -------------------------------

# Nihai sütunlar
final_columns = ['Symbol', 'Date', 'USD_Adj Close', 'USD_Close', 'USD_High', 'USD_Low', 'USD_Open', 'Volume']
timeseries_df = timeseries_df[final_columns]

# Tarih formatı son kontrol
timeseries_df['Date'] = pd.to_datetime(timeseries_df['Date'], errors='coerce').dt.date

# CSV dosyasını kaydet
timeseries_df.to_csv(timeseries_output_path, index=False)
print(f"✅ Time series data for top 100 companies saved to {timeseries_output_path}")


/var/folders/g1/lmg3w4rs5t7f4c_vswsjfrz80000gn/T/ipykernel_7916/3617233408.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forex_df = pd.read_csv(forex_path, index_col=0, parse_dates=True)
/var/folders/g1/lmg3w4rs5t7f4c_vswsjfrz80000gn/T/ipykernel_7916/3617233408.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forex_df.index = pd.to_datetime(forex_df.index, errors='coerce').date


✅ Metadata, Forex, and Currency Info data loaded successfully.
✅ Currency conversion applied successfully.
✅ Time series data for top 100 companies saved to /Users/erolberkiyibozkurt/Documents/GitHub/Python/Projects/The Influence of AI Boom on Tech Stocks/Background Data/stock_timeseries_top100.csv
